In [11]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk import PorterStemmer
import pickle

In [12]:
def removePunctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text   

In [13]:
vocab = pd.read_csv('../static/Model/vocabulary.txt',header=None)
tokens = vocab[0].tolist()

In [14]:
with open('../static/Model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [15]:
with open('../static/Model/model.pickle','rb') as f:
    model = pickle.load(f)

In [16]:
ps = PorterStemmer()

In [17]:
def preprocessing(text):
    data = pd.DataFrame([text],columns=['tweet'])
    
    data["tweet"] = data["tweet"].apply(lambda x:" ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x:" ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(removePunctuations)
    data["tweet"] = data["tweet"].str.replace(r'\d+','',regex=True)
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x :' '.join(ps.stem(x) for x in x.split()))

    return data['tweet']

In [18]:
def vectorizer(ds,vocabulary):
    vectorized_list = []

    for sentence in ds :
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)

    return np.asarray(vectorized_list,dtype=np.float32)

In [19]:
def predict(text):
    result = model.predict(text)
    if result == 0 :
        print("Positive")
    else:
        print("Negative")


In [20]:
txt = "nice one"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt,tokens)
predict(vectorized_txt)

Negative
